## Observations and Insights 

In [ ]:
##ADD IN NBVIEWER LINK HERE

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset

# Display the data table for preview


In [11]:
mouse_metadata

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16
...,...,...,...,...,...
244,z314,Stelasyn,Female,21,28
245,z435,Propriva,Female,12,26
246,z581,Infubinol,Female,24,25
247,z795,Naftisol,Female,13,29


In [10]:
study_results

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.000000,0
1,f932,0,45.000000,0
2,g107,0,45.000000,0
3,a457,0,45.000000,0
4,c819,0,45.000000,0
...,...,...,...,...
1888,r944,45,41.581521,2
1889,u364,45,31.023923,3
1890,p438,45,61.433892,1
1891,x773,45,58.634971,4


In [9]:
#Merge data along mouse id column, probably left join using study results as key? Try & See. 
merged_mice = pd.merge(study_results, mouse_metadata, how = 'left', on = 'Mouse ID', suffixes = ("_results", "_metadata"))
merged_mice

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,f932,0,45.000000,0,Ketapril,Male,15,29
2,g107,0,45.000000,0,Ketapril,Female,2,29
3,a457,0,45.000000,0,Ketapril,Female,11,30
4,c819,0,45.000000,0,Ketapril,Male,21,25
...,...,...,...,...,...,...,...,...
1888,r944,45,41.581521,2,Capomulin,Male,12,25
1889,u364,45,31.023923,3,Capomulin,Male,18,17
1890,p438,45,61.433892,1,Ceftamin,Female,11,26
1891,x773,45,58.634971,4,Placebo,Female,21,30


In [13]:
# Checking the number of mice.
mouse_count = merged_mice['Mouse ID'].nunique()
mouse_count

249

In [17]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
merged_mice_unique = merged_mice.drop_duplicates(subset = ['Mouse ID', 'Timepoint'], keep = "first").reset_index()
merged_mice_unique

,index,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,0,b128,0,45.000000,0,Capomulin,Female,9,22
1,1,f932,0,45.000000,0,Ketapril,Male,15,29
2,2,g107,0,45.000000,0,Ketapril,Female,2,29
3,3,a457,0,45.000000,0,Ketapril,Female,11,30
4,4,c819,0,45.000000,0,Ketapril,Male,21,25
...,...,...,...,...,...,...,...,...,...
1883,1888,r944,45,41.581521,2,Capomulin,Male,12,25
1884,1889,u364,45,31.023923,3,Capomulin,Male,18,17
1885,1890,p438,45,61.433892,1,Ceftamin,Female,11,26
1886,1891,x773,45,58.634971,4,Placebo,Female,21,30


In [32]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mice = merged_mice.duplicated(subset = ['Mouse ID', 'Timepoint'], keep = 'first')
duplicate_mice_true = duplicate_mice.value_counts()
duplicate_mice_true_count = duplicate_mice_true.tolist()
duplicate_mice_true_count = duplicate_mice_true_count[1]
duplicate_mice

0       False
1       False
2       False
3       False
4       False
        ...  
1888    False
1889    False
1890    False
1891    False
1892    False
Length: 1893, dtype: bool

In [63]:
#Find location of duplicate (copy) mice rows, put em in a list 
copy_mice = duplicate_mice[duplicate_mice == True].index
copy_mice = copy_mice.tolist()
copy_mice

[137, 360, 681, 869, 1111]

In [64]:
#Make a smaller df the duplicate mouse data 
#It's a list of each duplicate mouse for now, figure out how to make it a df later
copy_mice_data = []
for i in copy_mice:
    copy_mice_data.append(merged_mice.loc[i])
copy_mice_data

[Mouse ID                  g989
 Timepoint                    0
 Tumor Volume (mm3)          45
 Metastatic Sites             0
 Drug Regimen          Propriva
 Sex                     Female
 Age_months                  21
 Weight (g)                  26
 Name: 137, dtype: object,
 Mouse ID                  g989
 Timepoint                    5
 Tumor Volume (mm3)     47.5704
 Metastatic Sites             0
 Drug Regimen          Propriva
 Sex                     Female
 Age_months                  21
 Weight (g)                  26
 Name: 360, dtype: object,
 Mouse ID                  g989
 Timepoint                   10
 Tumor Volume (mm3)     49.8805
 Metastatic Sites             0
 Drug Regimen          Propriva
 Sex                     Female
 Age_months                  21
 Weight (g)                  26
 Name: 681, dtype: object,
 Mouse ID                  g989
 Timepoint                   15
 Tumor Volume (mm3)      53.442
 Metastatic Sites             0
 Drug Regimen          

In [69]:
merged_mice_unique.head()

,index,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,0,b128,0,45.0,0,Capomulin,Female,9,22
1,1,f932,0,45.0,0,Ketapril,Male,15,29
2,2,g107,0,45.0,0,Ketapril,Female,2,29
3,3,a457,0,45.0,0,Ketapril,Female,11,30
4,4,c819,0,45.0,0,Ketapril,Male,21,25


In [65]:
# Checking the number of mice in the clean DataFrame, called merged_mice_unique
mouseCountUnique = merged_mice_unique['Mouse ID'].count()
mouseCountUnique

1888

## Summary Statistics

In [73]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
describe_table = merged_mice_unique.describe()
mean_vol_by_treatment = merged_mice_unique.groupby('Drug Regimen').mean()['Tumor Volume (mm3)']
mean_vol_by_treatment = round(mean_vol_by_treatment, 2)

# This method is the most straighforward, creating multiple series and putting them all together at the end.



Drug Regimen
Capomulin    40.68
Ceftamin     52.59
Infubinol    52.88
Ketapril     55.24
Naftisol     54.33
Placebo      54.03
Propriva     52.39
Ramicane     40.22
Stelasyn     54.23
Zoniferol    53.24
Name: Tumor Volume (mm3), dtype: float64

In [76]:
median_vol_by_treatment = merged_mice_unique.groupby('Drug Regimen').median()['Tumor Volume (mm3)']
median_vol_by_treatment = round(median_vol_by_treatment, 2)
median_vol_by_treatment

Drug Regimen
Capomulin    41.56
Ceftamin     51.78
Infubinol    51.82
Ketapril     53.70
Naftisol     52.51
Placebo      52.29
Propriva     50.91
Ramicane     40.67
Stelasyn     52.43
Zoniferol    51.82
Name: Tumor Volume (mm3), dtype: float64

In [79]:
stdev_vol_by_treatment = merged_mice_unique.groupby('Drug Regimen').std()['Tumor Volume (mm3)']
stdev_vol_by_treatment = round(stdev_vol_by_treatment, 2)
stdev_vol_by_treatment

pandas.core.series.Series

In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [10]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
